## Community Erkennung 

Louvain: um hierarchische Community-Strukturen zu entdecken und dabei die Modularität zu maximieren. Er ist effizient in grossen Netzwerken <br>
Label Proagation: schneller und einfacher als Louvain und basiert auf der Idee, dass Knoten die Labels ihrer Nachbarn übernehmen. Er eignet sich besonders gut für Netzwerke, in denen die Community-Struktur dynamisch ist oder wo eine schnelle, vorläufige Einschätzung der Community-Struktur benötigt wird.

#### Cypher Louvain Query

In [4]:
from py2neo import Graph
graph = Graph("bolt://localhost:7687", auth=("neo4j", ""))

louvain_query = """
CALL gds.louvain.stream('graphAcc20')
YIELD nodeId, communityId
RETURN gds.util.asNode(nodeId).accountNumber AS account, communityId
ORDER BY communityId
"""
louvain_results = graph.run(louvain_query).data()

for result in louvain_results:
    print(result)


{'account': 'A2', 'communityId': 1}
{'account': 'A3', 'communityId': 1}
{'account': 'A1', 'communityId': 4}
{'account': 'A4', 'communityId': 4}
{'account': 'A5', 'communityId': 4}


Output Deutung:<br>
Community ID = Gemeinschaftsidentifikation

#### Übergabe an GPT

In [5]:
import openai

class OpenAIWrapper:
    def __init__(self, api_key):
        self.api_key = api_key
        openai.api_key = self.api_key

    def generate_chat_response(self, prompt, model="gpt-3.5-turbo", temperature=0.7, max_tokens=150):
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                max_tokens=max_tokens
            )
            return response.choices[0].message['content'].strip() 
        except Exception as e:
            return f"An error occurred: {str(e)}"

In [6]:
openai_wrapper = OpenAIWrapper(api_key='')
prompt_intro = "Based on the Louvain Community Detection algorithm, the nodes in the network have been assigned to different communities. Here are the nodes grouped by their respective communities:\n\n"
prompt_body = "\n".join([f"- {res['account']} in community {res['communityId']}" for res in louvain_results[:10]])  # Top 10 
prompt_topic = "\n\nGiven that these communities may represent different clusters of entities within the network, it's important to analyze if any of these patterns indicate potential money laundering activities. The identification of suspicious patterns and the subsequent decision on whether they constitute money laundering activities require careful consideration of various factors, including transaction patterns, individuals involved, and community structures."
prompt_question = "\n\nWhat patterns or insights can be observed from the community structure identified by the Louvain algorithm? How do these communities reflect the underlying relationships or dynamics within the network?"

complete_prompt = prompt_intro + prompt_body + prompt_question

print(complete_prompt)

response = openai_wrapper.generate_chat_response(complete_prompt)
print(f"GPT-3's Insights: {response}")


Based on the Louvain Community Detection algorithm, the nodes in the network have been assigned to different communities. Here are the nodes grouped by their respective communities:

- A2 in community 1
- A3 in community 1
- A1 in community 4
- A4 in community 4
- A5 in community 4

What patterns or insights can be observed from the community structure identified by the Louvain algorithm? How do these communities reflect the underlying relationships or dynamics within the network?
GPT-3's Insights: An error occurred: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.


#### Cypher Proagation Query

In [7]:
label_propagation_query = """
CALL gds.labelPropagation.stream('graphAcc20')
YIELD nodeId, communityId
RETURN gds.util.asNode(nodeId).accountNumber AS account, communityId
ORDER BY communityId
"""
label_propagation_results = graph.run(label_propagation_query).data()

for result in label_propagation_results:
    print(result)


{'account': 'A1', 'communityId': 6}
{'account': 'A2', 'communityId': 6}
{'account': 'A3', 'communityId': 6}
{'account': 'A4', 'communityId': 6}
{'account': 'A5', 'communityId': 6}


Output Deutung: <br>
Community ID = Gemeiscnaftszuordnung

#### Übergabe an GPT

In [8]:
prompt_intro = "Based on the label_propagation algorithm, the nodes in the network have been assigned to different communities. Here are the nodes grouped by their respective communities:\n\n"
prompt_body = "\n".join([f"- {res['account']} in community {res['communityId']}" for res in label_propagation_results[:10]])  # Top 10 
prompt_topic = "\n\nGiven that these communities may represent different clusters of entities within the network, it's important to analyze if any of these patterns indicate potential money laundering activities. The identification of suspicious patterns and the subsequent decision on whether they constitute money laundering activities require careful consideration of various factors, including transaction patterns, individuals involved, and community structures."
prompt_question = "\n\nWhat patterns or insights can be observed from the community structure identified by the Louvain algorithm? How do these communities reflect the underlying relationships or dynamics within the network?"

complete_prompt = prompt_intro + prompt_body + prompt_question

print(complete_prompt)

response = openai_wrapper.generate_chat_response(complete_prompt)
print(f"GPT-3's Insights: {response}")

Based on the label_propagation algorithm, the nodes in the network have been assigned to different communities. Here are the nodes grouped by their respective communities:

- A1 in community 6
- A2 in community 6
- A3 in community 6
- A4 in community 6
- A5 in community 6

What patterns or insights can be observed from the community structure identified by the Louvain algorithm? How do these communities reflect the underlying relationships or dynamics within the network?
GPT-3's Insights: An error occurred: You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.
